In [40]:
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup as soup
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
import pandas as pd
import time
from datetime import datetime, timedelta
import time
from IPython.display import clear_output
from config import conn_host, conn_database, conn_user, conn_password
import mysql.connector
from dateutil import tz

In [41]:
months = dict(Jan=1,Feb=2,Mar=3,Apr=4,May=5,Jun=6,Jul=7,Aug=8,Sep=9,Oct=10,Nov=11,Dec=12)

# If you want to change the league, browse through the odds portal website to find the correct url parameters for you
country = "brazil"
league = "serie-a"

In [42]:
def connect_to_db():
    return mysql.connector.connect(host=conn_host, 
                                     database=conn_database,
                                     user=conn_user,
                                     password=conn_password)

def execute_query(query, read_only = True):
    resp = None
    try:
        db = connect_to_db()
        if read_only:
            resp = pd.read_sql_query(query, db)
        else:
            mycursor = db.cursor()
            mycursor.execute(query)

            db.commit()
    except Exception as e:
        print(e)
    db.close()
    return resp

In [43]:
execute_query("ALTER TABLE matches " +
              "ADD home_odds FLOAT NULL," +
              "ADD away_odds FLOAT NULL," +
              "ADD draw_odds FLOAT NULL;")

Execution failed on sql 'ALTER TABLE matches ADD home_odds FLOAT NULL,ADD away_odds FLOAT NULL,ADD draw_odds FLOAT NULL;': 1060 (42S21): Duplicate column name 'home_odds'


C:\Users\caiog\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [53]:
teams = execute_query("SELECT * FROM teams")

C:\Users\caiog\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [54]:
print(teams)

        id            name
0       33  Manchester Utd
1       34       Newcastle
2       35     Bournemouth
3       36          Fulham
4       37    Huddersfield
..     ...             ...
243  12277        Ipatinga
244  13173      Chivas USA
245  13974      Santa Cruz
246  16489       Austin FC
247  18310       Charlotte

[248 rows x 2 columns]


In [55]:
def get_betting_odds(season):
    from_zone = tz.gettz('UTC')
    to_zone = tz.gettz('America/Sao_Paulo')
    if '2014' in season and league == "j1-league": # Different name for J1 league for the 2014 season
        base_url = f"https://www.oddsportal.com/soccer/{country}/j-league{season}/results/"
    else:
        base_url = f"https://www.oddsportal.com/soccer/{country}/{league}{season}/results/"
    option = Options()
    option.headless = True
    driver = webdriver.Chrome(ChromeDriverManager().install(), options=option)
    driver.get(base_url)
    time.sleep(5)
    
    try:
        button = driver.find_element("xpath", '/html/body/div[3]/div[3]/div/div[1]/div/div[2]/div/button[1]')
        button.click()
    except:
        print('A problem occoured while trying to close the cookies popup')
        
    button = driver.find_element("xpath", '/html/body/div[1]/div/div[2]/div[6]/div[1]/div/div[1]/div[2]/div[1]/div[6]/div/a[last()]')
    button.click()
    time.sleep(2)
    
    element = driver.find_element('id', "tournamentTable")
    htmlContent = element.get_attribute('outerHTML')
    page_soup = soup(htmlContent, "html.parser")
    active_page = int(''.join(page_soup.find('span', {"class": "active-page"}).findAll(text=True)))
    print('{}: Page {}'.format(season[1:], active_page))
    
    games = []
    
    while int(active_page) >= 1:
        dates = page_soup.findAll('tr', {"class": "center nob-border"})
        games_registered = 0
        
        for i in range(len(dates), 0, -1):
            try:
                if len(dates) == i:
                    date_games = dates[i-1].find_next_siblings('tr', {"class": "deactivate"})
                else:
                    date_games = dates[i-1].find_next_siblings('tr', {"class": "deactivate"})[:-1*games_registered]
                games_registered += len(date_games)
                date_info_splitted = dates[i-1].contents[0].text.split('-')
                if len(date_info_splitted) > 1 and 'stage' not in date_info_splitted[1]: # Include named stages that are not playoffs
                    continue
                date_text = date_info_splitted[0].strip()
                for g in date_games:
                    try:
                        unix_date = datetime(int(date_text.split(' ')[2]), 
                                                        months[date_text.split(' ')[1]], 
                                                        int(date_text.split(' ')[0]), 
                                                        hour=int(g.contents[0].text.strip().split(':')[0]), 
                                                        minute=int(g.contents[0].text.strip().split(':')[1]), 
                                                        second=0).replace(tzinfo=from_zone).astimezone(to_zone)
                        home_team_string = g.contents[1].text.split(' - ')[0].strip()
                        away_team_string = g.contents[1].text.split(' - ')[1].strip()
                        home_team = teams.loc[teams['name'].str.lower() == home_team_string.lower()].iloc[0]
                        away_team = teams.loc[teams['name'].str.lower() == away_team_string.lower()].iloc[0]
                        games_parsed = [unix_date,
                                          home_team['id'], # Home Team Id
                                          away_team['id'], # Away Team Id
                                          home_team['name'], # Team A Name
                                          away_team['name'], # Team B Name
                                          g.contents[2].text.split(':')[0].strip(), # Team A Score
                                          g.contents[2].text.split(':')[1].replace('OT', '').strip(), # Team B Score
                                          g.contents[3].text,  # Team A Odds
                                          g.contents[4].text,  # Draw Odds
                                          g.contents[5].text]  # Team B Odds
                        games.append(games_parsed)
                    except:
                        continue
                # print('{} Games appended'.format(len(games_parsed)))
            except Exception as e:
                print(e)
                continue
        
        btn_next_page = driver.find_element("xpath", '/html/body/div[1]/div/div[2]/div[6]/div[1]/div/div[1]/div[2]/div[1]/div[6]/div/a[2]')
        btn_next_page.click()
        time.sleep(4)
        
        element = driver.find_element('id', "tournamentTable")
        htmlContent = element.get_attribute('outerHTML')
        page_soup = soup(htmlContent, "html.parser")
        active_page -= 1
        clear_output(wait=True)
        print('{}: Page {}'.format(season[1:], active_page))
    
    driver.quit()
    return games
    

In [56]:
season = 2018
games = []
while season < 2022:
    games.extend(get_betting_odds(f'-{season}')) # Brazil/Same year leagues
#     games.extend(get_betting_odds(f'-{season}-{season+1}')) # European/Two year leagues leagues

    season += 1

games.extend(get_betting_odds('')) # For the current season

odds_df = pd.DataFrame(games, columns=['GAME_DATE', 'TEAM_A_ID', 'TEAM_B_ID', 'TEAM_A', 'TEAM_B', 'TEAM_A_SCORE', 'TEAM_B_SCORE', 'TEAM_A_ODDS', 'DRAW_ODDS', 'TEAM_B_ODDS'])
print(len(games))

: Page 0
1044


In [57]:
odds_df.head()

,GAME_DATE,TEAM_A_ID,TEAM_B_ID,TEAM_A,TEAM_B,TEAM_A_SCORE,TEAM_B_SCORE,TEAM_A_ODDS,DRAW_ODDS,TEAM_B_ODDS
0,2018-02-02 22:00:00-02:00,2994,2317,U. Catolica,Deportes Temuco,2,1,1.60,3.81,5.48
1,2018-02-04 19:30:00-02:00,2318,2326,Palestino,Union La Calera,1,2,2.14,3.31,3.28
2,2018-02-04 14:00:00-02:00,2327,2315,Antofagasta,Colo Colo,1,2,3.69,3.49,1.93
3,2018-02-04 22:30:00-02:00,2329,2328,A. Italiano,Huachipato,2,4,2.00,3.41,3.59
4,2018-02-05 20:00:00-02:00,2323,2321,U. de Chile,U. Espanola,1,2,1.87,3.48,4.01


In [58]:
game_errors = []
for index, row in odds_df.iterrows():
    try:
        date_minus_one_day = row['GAME_DATE'] - timedelta(days=1)
        date_plus_one_day = row['GAME_DATE'] + timedelta(days=1)
        game_id = execute_query(f"SELECT id from matches WHERE (DATE_FORMAT(date,'%d/%m/%Y') = DATE_FORMAT('{row['GAME_DATE']}','%d/%m/%Y') OR DATE_FORMAT(date,'%d/%m/%Y') = DATE_FORMAT('{date_minus_one_day}','%d/%m/%Y') OR DATE_FORMAT(date,'%d/%m/%Y') = DATE_FORMAT('{date_plus_one_day}','%d/%m/%Y')) AND home_id = {row['TEAM_A_ID']} AND away_id = {row['TEAM_B_ID']}").iloc[0]['id']
        update_query = f"UPDATE matches SET home_odds = {row['TEAM_A_ODDS']}, away_odds = {row['TEAM_B_ODDS']}, draw_odds = {row['DRAW_ODDS']} WHERE id = {game_id}"
        execute_query(update_query, False)
    except Exception as e:
        new_item = dict()
        new_item['error'] = e
        new_item['match'] = row
        game_errors.append(new_item)
    clear_output(wait=True)
    print(f"{index}/{len(odds_df.index)}")

1043/1044


In [59]:
if len(game_errors) == 0:
    print('There were no errors while reading games from the Odds Portal website.')
else:
    print('The following games had an error while trying to read games from the Odds Portal website:')
    for item in game_errors:
        print(f"\n{item['match']['GAME_DATE']}: {item['match']['TEAM_A']}({item['match']['TEAM_A_ID']}) {item['match']['TEAM_A_SCORE']} x {item['match']['TEAM_B_SCORE']} {item['match']['TEAM_B']}({item['match']['TEAM_B_ID']})")
        print(f"ERROR: {item['error']}")

The following games had an error while trying to read games from the Odds Portal website:

2018-02-05 14:00:00-02:00: Everton(45) 2 x 3 Deportes Iquique(2319)
ERROR: single positional indexer is out-of-bounds

2018-02-17 14:00:00-02:00: Everton(45) 0 x 1 U. Catolica(2994)
ERROR: single positional indexer is out-of-bounds

2018-02-25 20:30:00-03:00: A. Italiano(2329) 1 x 0 Everton(45)
ERROR: single positional indexer is out-of-bounds

2018-03-02 20:00:00-03:00: Everton(45) 2 x 1 San Luis(2322)
ERROR: single positional indexer is out-of-bounds

2018-03-17 12:00:00-03:00: Everton(45) 0 x 1 U. de Chile(2323)
ERROR: single positional indexer is out-of-bounds

2018-03-31 15:30:00-03:00: Huachipato(2328) 2 x 0 Everton(45)
ERROR: single positional indexer is out-of-bounds

2018-04-07 12:00:00-03:00: Everton(45) 1 x 3 Union La Calera(2326)
ERROR: single positional indexer is out-of-bounds

2018-04-14 12:00:00-03:00: Palestino(2318) 2 x 1 Everton(45)
ERROR: single positional indexer is out-of-bo


2022-02-11 20:30:00-03:00: Everton(45) 2 x 0 Coquimbo(2330)
ERROR: single positional indexer is out-of-bounds

2022-02-18 20:30:00-03:00: U. Espanola(2321) 1 x 1 Everton(45)
ERROR: single positional indexer is out-of-bounds

2022-02-25 20:00:00-03:00: Union La Calera(2326) 1 x 1 Everton(45)
ERROR: single positional indexer is out-of-bounds

2022-03-05 20:30:00-03:00: Everton(45) 1 x 1 Curico Unido(2316)
ERROR: single positional indexer is out-of-bounds

2022-03-12 18:00:00-03:00: U. Catolica(2994) 0 x 1 Everton(45)
ERROR: single positional indexer is out-of-bounds

2022-03-20 20:30:00-03:00: Everton(45) 0 x 0 A. Italiano(2329)
ERROR: single positional indexer is out-of-bounds

2022-04-01 17:30:00-03:00: La Serena(2341) 1 x 3 Everton(45)
ERROR: single positional indexer is out-of-bounds

2022-04-09 16:00:00-03:00: Everton(45) 0 x 1 Huachipato(2328)
ERROR: single positional indexer is out-of-bounds

2022-04-18 19:00:00-03:00: Nublense(2337) 2 x 1 Everton(45)
ERROR: single positional ind

In [60]:
fixtures_df = execute_query(f"SELECT m.id, m.date, m.season, l.id AS league_id, l.name AS league, ht.name as home_team, at.name as away_team, m.home_score, m.away_score, m.home_odds, m.away_odds, m.draw_odds FROM matches AS m INNER JOIN teams AS ht ON (m.home_id = ht.id) INNER JOIN teams AS at ON (m.away_id = at.id) INNER JOIN leagues AS l ON (m.league_id = l.id) WHERE m.season = 2022 and league_id = 72 ORDER BY m.date ASC")

C:\Users\caiog\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [61]:
fixtures_df.tail()

,id,date,season,league_id,league,home_team,away_team,home_score,away_score,home_odds,away_odds,draw_odds
246,838864,2022-08-20 11:00:00,2022,72,Serie B (Brazil),Ponte Preta,Guarani,1,0,NaN,NaN,NaN
247,838873,2022-08-20 16:30:00,2022,72,Serie B (Brazil),Chapecoense-SC,Brusque,1,0,NaN,NaN,NaN
248,838871,2022-08-20 19:00:00,2022,72,Serie B (Brazil),Sampaio Correa,CRB,1,2,NaN,NaN,NaN
249,838870,2022-08-21 16:00:00,2022,72,Serie B (Brazil),Gremio,Cruzeiro,2,2,NaN,NaN,NaN
250,838876,2022-08-23 21:30:00,2022,72,Serie B (Brazil),Sport Recife,Chapecoense-SC,1,0,NaN,NaN,NaN
